In [2]:
import re
import multiprocessing
from tqdm import tqdm

import pandas as pd
from wordcloud import WordCloud, STOPWORDS
from scipy.stats import norm
from gensim.models import word2vec
# from kaggle.competitions import twosigmanews

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

import numpy as np

%matplotlib inline

In [3]:
cpu_count = 2*multiprocessing.cpu_count()-1
print('Number of CPUs: {}'.format(cpu_count))

Number of CPUs: 55


In [4]:
news_df = pd.read_csv("news_train_df.csv")

In [22]:
america_df = pd.read_csv("America.csv")

In [99]:
america_df["time"] = pd.to_datetime(america_df["time"])
news_df["time"] = pd.to_datetime(news_df["time"])

In [100]:
america_df.head()

,time,close,open,high,low,volume,percent,code
0,2015-01-05,106.25,108.29,108.65,105.41,64.29M,-2.82%,AAPL
1,2015-01-06,106.26,106.54,107.43,104.63,65.80M,0.01%,AAPL
2,2015-01-07,107.75,107.20,108.20,106.69,40.11M,1.40%,AAPL
3,2015-01-08,111.89,109.23,112.15,108.70,59.36M,3.84%,AAPL
4,2015-01-09,112.01,112.67,113.25,110.21,53.70M,0.11%,AAPL


In [101]:
america_df['code'].unique()

array(['AAPL', 'V', 'BABA', 'ADBE', 'SNE'], dtype=object)

In [102]:
news_df = news_df[['assetCodes', 
                    'assetName', 
                    'sentimentClass', 
                    'time',
                    'sentimentNegative',
                    'sentimentNeutral',
                    'sentimentPositive']]

In [103]:
news_df['sentimentRange'] = np.where(
    news_df['sentimentNeutral'] > abs(
        news_df['sentimentPositive']-news_df['sentimentNegative']),
    0,
    news_df['sentimentPositive']-news_df['sentimentNegative']
)

In [125]:
news_test_df = news_df[ (news_df['time']>='2015-01-01')]
news_train_df = news_df[ (news_df['time']<'2015-01-01')]

In [126]:
news_test_df['time'] = pd.to_datetime(news_test_df['time'].dt.date)

/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [127]:
news_test_df['assetCodes'] = news_test_df['assetCodes'].map(flattenAssetCodes)

/home/yz4499/miniconda3/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [187]:
news_test_df.head()

,assetCodes,assetName,sentimentClass,time,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentRange
7274526,{LONG},eLong Inc,-1,2015-01-01,0.809405,0.129914,0.060681,-0.748724
7274527,{EXPE},Expedia Inc,-1,2015-01-01,0.819121,0.125242,0.055638,-0.763483
7274528,{SYX},Systemax Inc,1,2015-01-01,0.034652,0.122020,0.843328,0.808676
7274529,{SYX},Systemax Inc,1,2015-01-01,0.097430,0.156117,0.746453,0.649023
7274530,{BRO},Brown & Brown Inc,-1,2015-01-01,0.552344,0.248000,0.199656,-0.352688


In [179]:
america_df["sentimentRange"] = np.nan

In [180]:
for index1, row1 in america_df.iterrows():
    for index2, row2 in news_test_df[(news_test_df['time']==row1['time'])].iterrows():
        if row1['code'] in row2['assetCodes']:
            if np.isnan(row1['sentimentRange']):
                row1['sentimentRange'] = 0
            row1['sentimentRange'] += row2['sentimentRange']
    america_df.at[index1,'sentimentRange'] = row1['sentimentRange']
#     print(index1)
    if index1 > 0 and index1 % 100 == 0:
        print("%d/%d"%(index1, len(america_df)))

100/3772
200/3772
300/3772
400/3772
500/3772
600/3772
700/3772
800/3772
900/3772
1000/3772
1100/3772
1200/3772
1300/3772
1400/3772
1500/3772
1600/3772
1700/3772
1800/3772
1900/3772
2000/3772
2100/3772
2200/3772
2300/3772
2400/3772
2500/3772
2600/3772
2700/3772
2800/3772
2900/3772
3000/3772
3100/3772
3200/3772
3300/3772
3400/3772
3500/3772
3600/3772
3700/3772


In [193]:
america_df = america_df.fillna(0)

In [194]:
america_df.tail(50)

,time,close,open,high,low,volume,percent,code,sentimentRange
3722,2017-10-19,37.44,37.28,37.45,37.03,784.17K,-0.05%,SNE,0.0
3723,2017-10-20,37.55,37.48,37.68,37.42,557.14K,0.29%,SNE,0.0
3724,2017-10-23,37.51,37.66,37.77,37.48,366.08K,-0.11%,SNE,0.0
3725,2017-10-24,37.70,37.54,37.71,37.45,323.42K,0.51%,SNE,0.0
3726,2017-10-25,37.16,37.42,37.48,36.93,515.93K,-1.43%,SNE,0.0
3727,2017-10-26,37.86,37.59,37.98,37.51,817.45K,1.88%,SNE,0.0
3728,2017-10-27,37.83,37.70,37.85,37.59,559.18K,-0.08%,SNE,0.0
3729,2017-10-30,38.85,37.93,39.07,37.83,2.20M,2.70%,SNE,0.0
3730,2017-10-31,43.41,43.21,45.25,42.90,6.81M,11.74%,SNE,0.0
3731,2017-11-01,43.54,43.57,43.94,43.33,2.29M,0.30%,SNE,0.0


In [195]:
america_df.to_csv("AmericaWithNews.csv")

In [71]:
import ast
def flattenAssetCodes(codes):
    lst = ast.literal_eval(codes)
    def get1st(code):
        return code.split('.')[0]
    return set(map(get1st, lst))

In [70]:
strtst = "{'LONG.L', 'LONG.O', 'AAPL', 'KK.O'}"
flattenAssetCodes(strtst)

{'AAPL', 'KK', 'LONG'}